In [ ]:
!git clone https://github.com/triablomanon/tgbn-project.git

Cloning into 'tgbn-project'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 79 (delta 44), reused 72 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 90.13 KiB | 4.10 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
%cd tgbn-project

/content/tgbn-project


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /content/drive/MyDrive/tgbn_cache
!ln -s /content/drive/MyDrive/tgbn_cache saved_labeled_node_interaction_indices

In [ ]:
!pip install pandas -q
import torch
print(torch.__version__)
!pip install torch-geometric -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install py-tgb -q
!pip install modules


2.8.0+cu126
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2

In [ ]:
!python dygformer/test_labels.py
# The output of this cell shows us that labels are a probability distribution over the music genres

Loading tgbn-genre dataset...
[################################] 150456/150456 - 00:00:45
17858396it [01:00, 293819.13it/s]
2741936it [00:06, 452059.22it/s]

=== Dataset Info ===
Number of classes (genres): 513
Number of timestamps with labels: 1579
Total labeled nodes (across all timestamps): 256493

=== Examining First 10 Labels ===

Label #1:
  Timestamp: 1108443600
  Node ID: 533
  Label shape: (513,)
  Number of active genres: 33
  Active genre indices: [ 1  2  4  5  7  8 18 20 22 24 29 43 49 51 53 54 56 58 66 72]...
  Label values at active positions: [0.0362972  0.00566081 0.11384996 0.0668751  0.01886783 0.00524037
 0.01665354 0.01200711 0.04967586 0.01532975 0.0112734  0.01434797
 0.00808153 0.17058364 0.08572021 0.01091456 0.07216231 0.00436233
 0.02146456 0.0268307 ]
  Sum of label: 1.00

Label #2:
  Timestamp: 1108443600
  Node ID: 519
  Label shape: (513,)
  Number of active genres: 12
  Active genre indices: [  2   5  16  18  20  23  46  49  93  94 192 194]...
  Label val

# DyGFormer

In [ ]:
!python dygformer/train_node_classification.py \
    --dataset_name tgbn-genre \
    --model_name DyGFormer \
    --load_best_configs \
    --subset_fraction 0.1 \
    --timestamp_threshold 1135778006 \
    --filter_seed 42 \
    --num_runs 5 \
    --gpu 0

After filtering: 1667637 interactions (from 17858395 original)
Selected 99 source nodes out of 992 (10.0%)
Set edge IDs to array indices: [0, 1667636]
100% 1579/1579 [00:00<00:00, 1993.43it/s]
Loaded preprocessed cache from ./saved_labeled_node_interaction_indices/tgbn-genre_subset0.1_ts1135778006.0_seed42.npy
100% 25461/25461 [00:00<00:00, 164097.20it/s]
[FILTERED] Re-indexed 609 nodes:
  - src_node_ids from [523, 1490] to [510, 608]
  - dst_node_ids from [0, 512] to [0, 509]
[DEBUG] edge_ids range: [1, 1667637]
[DEBUG] edge_raw_features shape: (1667638, 1), max valid index: 1667637
[DEBUG] src_node_ids range: [511, 609]
[DEBUG] node_raw_features shape: (611, 1), max valid index: 610
The dataset has 1667637 interactions, involving 609 different nodes
The training dataset has 1185239 interactions, involving 592 different nodes
The validation dataset has 240194 interactions, involving 575 different nodes
The test dataset has 242204 interactions, involving 581 different nodes
INFO:root:*

# DyGFormer with MA Features

In [ ]:
!python dygformer/train_node_classification.py \
    --dataset_name tgbn-genre \
    --model_name DyGFormer \
    --gpu 0 \
    --load_best_configs \
    --use_ma_features \
    --ma_window_size 7 \
    --subset_fraction 0.1 \
    --timestamp_threshold 1135778006 \
    --filter_seed 42 \
    --num_epochs 100 \
    --patience 20

[################################] 150456/150456 - 00:00:44
17858396it [01:03, 281044.37it/s]
2741936it [00:07, 373423.85it/s]
After filtering: 1667637 interactions (from 17858395 original)
Selected 99 source nodes out of 992 (10.0%)
Set edge IDs to array indices: [0, 1667636]
100% 1579/1579 [00:00<00:00, 2074.57it/s]
Loaded preprocessed cache from ./saved_labeled_node_interaction_indices/tgbn-genre_subset0.1_ts1135778006.0_seed42.npy
100% 25461/25461 [00:00<00:00, 174782.40it/s]
[FILTERED] Re-indexed 609 nodes:
  - src_node_ids from [523, 1490] to [510, 608]
  - dst_node_ids from [0, 512] to [0, 509]
[DEBUG] edge_ids range: [1, 1667637]
[DEBUG] edge_raw_features shape: (1667638, 1), max valid index: 1667637
[DEBUG] src_node_ids range: [511, 609]
[DEBUG] node_raw_features shape: (611, 1), max valid index: 610
The dataset has 1667637 interactions, involving 609 different nodes
The training dataset has 1185239 interactions, involving 592 different nodes
The validation dataset has 240194 